In [1]:
#Imports
import pandas as pd
import os
import tensorflow as tf
from keras import backend as K

from utils.modelgenerator import *
from utils.modelhandler import *
from utils.datahandler import *

#Get data 
cwd = os.path.normpath(os.getcwd())
df = pd.read_csv(cwd+'/data/df_with_final_features.csv', index_col='Date') #df = pd.read_csv('user5.csv')
df.index = pd.to_datetime(df.index)
#df = df[['User5', 'temp', 'rhum']]
df.fillna(0, inplace=True)

df_array = []
for idx in range(30):
    df_array.append(df[[f'User{idx+1}', 'temp', 'rhum', 'wspd', 'PC1', 'hour sin', 'hour cos', f'User{idx+1}_lag_24hrs']])

#df_array[3].head(3)

#Train, Validation and Test datasets
sequence_length = 25
batch_size = 16
num_features = df_array[0].shape[1]

dh = Datahandler()

X_train, y_train, X_val, y_val, X_test, y_test = {}, {}, {}, {}, {}, {}

#Create Train, Validation and Test datasets
for idx, df in enumerate(df_array):
    n = len(df)
    train_df = df[0:int(n*0.7)]
    val_df = df[int(n*0.7):int(n*0.9)]
    test_df = df[int(n*0.9):]

    # Min max sclaing
    train_df = dh.min_max_scaling(train_df)
    val_df = dh.min_max_scaling(val_df)
    test_df = dh.min_max_scaling(test_df)

    # Sequencing
    train_sequences = dh.create_sequences(train_df, sequence_length)
    val_sequences = dh.create_sequences(val_df, sequence_length)
    test_sequences = dh.create_sequences(test_df, sequence_length)

    #Split into feature and label
    X_train[f'user{idx+1}'], y_train[f'user{idx+1}'] = dh.prepare_data(train_sequences, batch_size)
    X_val[f'user{idx+1}'], y_val[f'user{idx+1}'] = dh.prepare_data(val_sequences, batch_size)
    X_test[f'user{idx+1}'], y_test[f'user{idx+1}'] = dh.prepare_data(test_sequences, batch_size)

#General Hyperparameters
# #All models
horizon = 1
max_epochs = 100
m1 = ModelGenerator()
mh = Modelhandler()

loss = tf.keras.losses.MeanSquaredError()
metrics=[
    tf.keras.metrics.RootMeanSquaredError(), 
    tf.keras.metrics.MeanAbsolutePercentageError(),
    tf.keras.metrics.MeanAbsoluteError(),
]

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=10,mode='min')
timing_callback = TimingCallback()
custom_callback = CustomCallback()
#model_checkpoint = ModelCheckpoint('models/best_model.h5', save_best_only=True, monitor='val_loss', mode='min')
callbacks=[early_stopping, timing_callback, custom_callback] #model_checkpoint


# Federated Learning mechanisms

In [2]:
#Average Aggregation
def avg_weights(weight_list):
    """
    Return the sum of the listed weights. The is equivalent to avg of the weights
    """
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*weight_list):
        layer_mean = tf.math.reduce_mean(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
        
    return avg_grad

In [3]:
# Clipped Average Aggregation
def clipped_avg_weights(weight_list, clip_threshold=None):
    #clip_threshold = 1, 1.5, 0.5, 2
    """
    Return the clipped average of the listed weights.
    
    Args:
        weight_list (list of list of tensors): List of weights from clients.
        clip_threshold (float or None): Optional threshold for gradient clipping.
            If None, no clipping is performed.

    Returns:
        avg_grad (list of tensors): List of clipped average weights.
    """
    avg_grad = list()
    
    # Get the average gradient across all client gradients
    for grad_list_tuple in zip(*weight_list):
        layer_mean = tf.math.reduce_mean(grad_list_tuple, axis=0)
        
        # Optionally clip the gradients
        if clip_threshold is not None:
            layer_mean = tf.clip_by_value(layer_mean, -clip_threshold, clip_threshold)
        
        avg_grad.append(layer_mean)
        
    return avg_grad


In [4]:
#Differential Privacy Average Aggregation / Adding noise
def avg_weights_with_noise(weight_list, clip_threshold=None, noise_scale=0.02):
    """
    Return the clipped average of the listed weights with added random noise.

    Args:
        weight_list (list of list of tensors): List of weights from clients.
        clip_threshold (float or None): Optional threshold for gradient clipping.
            If None, no clipping is performed.
        noise_scale (float): Scale factor for the random noise. Higher values introduce more noise.

    Returns:
        avg_grad (list of tensors): List of clipped average weights with added noise.
    """
    avg_grad = list()

    # Get the average gradient across all client gradients
    for grad_list_tuple in zip(*weight_list):
        layer_mean = tf.math.reduce_mean(grad_list_tuple, axis=0)

        # Optionally clip the gradients
        if clip_threshold is not None:
            layer_mean = tf.clip_by_value(layer_mean, -clip_threshold, clip_threshold)

        # Add random noise to the gradients
        noise = tf.random.normal(shape=layer_mean.shape, mean=0.0, stddev=noise_scale)
        noisy_layer_mean = layer_mean + noise
        avg_grad.append(noisy_layer_mean)

    return avg_grad


In [5]:
def avg_weights_with_noise_fedprox(weight_list, clip_threshold=None, noise_scale=0.001, proximal_term=0.1):
    avg_grad = list()

    for grad_list_tuple in zip(*weight_list):
        layer_mean = tf.math.reduce_mean(grad_list_tuple, axis=0)

        if clip_threshold is not None:
            layer_mean = tf.clip_by_value(layer_mean, -clip_threshold, clip_threshold)

        noise = tf.random.normal(shape=layer_mean.shape, mean=0.0, stddev=noise_scale)
        noisy_layer_mean = layer_mean + noise

        # Add FedProx proximal term
        proximal_update = -proximal_term * noisy_layer_mean

        avg_grad.append(noisy_layer_mean + proximal_update)

    return avg_grad


# Cluster Selection

In [6]:
num_clusters = 4
y = np.loadtxt(f'evaluations/federated_learning/clusters_KMeans{num_clusters}_dtw.csv', delimiter=',').astype(int)

cluster_users = {i: [] for i in range(num_clusters)}

# Iterate through each cluster
for cluster_number in range(num_clusters):
    users_in_cluster = np.where(y == cluster_number)[0] +1
    cluster_users[cluster_number] = users_in_cluster
cluster_users

{0: array([ 1,  2,  4,  5,  7, 10, 11, 12, 15, 16, 17, 21, 22, 23, 24, 25, 28,
        30], dtype=int64),
 1: array([29], dtype=int64),
 2: array([ 6, 13, 18, 20, 26], dtype=int64),
 3: array([ 3,  8,  9, 14, 19, 27], dtype=int64)}

# Federated Learning with Mixture of Experts Model

In [7]:
#Testing
# Cluster 4, 6, 8, 10, 15
# Cluster Algorithm: K-Menas, Self organizing maps
# Federated Algorithm: Clipped, momentum, noise

In [8]:
results = pd.DataFrame(columns=['architecture', 'train_time', 'avg_time_epoch', 'mse','mse_std', 'rmse','rmse_std','mape','mape_std','mae','mae_std'])

all_results = pd.DataFrame(columns=["user", "architecture", "train_time", "avg_time_epoch", "mse", "rmse", "mape", "mae"])

In [9]:
federated_rounds = 3

model_architecture = "Dense_MoE_e4d8_d16"
dense_units = 16
num_experts = 4
expert_units = 8

# Create global models for each cluser (6)
for cluster in range(num_clusters):
#Build and save global model
    global_model = m1.build_soft_dense_moe_model(X_train[f'user{1}'], batch_size, horizon, dense_units, expert_units, num_experts, m1)
    global_model.save(cwd + f"/models/FL/Testing/cluster_{cluster}/{model_architecture}/FederatedRound{0}")

  

for federated_round  in range(federated_rounds):
    print("Started Federated training round ----------", federated_round+1, f"/ {federated_rounds}")

    for cluster_number, users_in_cluster in cluster_users.items():
        print(f"Cluster {cluster_number}:")

        #Get global models weights
        global_model = keras.models.load_model(cwd + f"/models/FL/Testing/cluster_{cluster_number}/{model_architecture}/Federatepund{federated_round}", compile=False)
        global_model_weights = global_model.get_weights()

        #initial list for local model weights
        local_weight_list = list()


        #for idx, user in enumerate(df_array): 
        for user_index in users_in_cluster:
            user_df = df_array[user_index-1]  # Get the user's DataFrame from the array
            print(f"User {user_index}") 
                      
            #build and compile local model X_train, batch_size, horizon, dense_units,  expert_units, num_experts, m1
            local_model = m1.build_soft_dense_moe_model(X_train[f'user{user_index}'], batch_size, horizon, dense_units, expert_units, num_experts, m1)
            local_model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=0.0006), metrics=metrics)

            #set local model weight to the weight of the global model
            local_model.set_weights(global_model_weights)
            
            #Fit local model to local data
            histroy, user_results = mh.compile_fit_evaluate_model(
                model=local_model, 
                loss=loss, 
                metrics=metrics, 
                X_train=X_train[f'user{user_index}'],
                y_train = y_train[f'user{user_index}'], 
                max_epochs = max_epochs, 
                batch_size=batch_size, 
                X_val=X_val[f'user{user_index}'], 
                y_val=y_val[f'user{user_index}'], 
                X_test=X_test[f'user{user_index}'], 
                y_test=y_test[f'user{user_index}'], 
                callbacks=callbacks, 
                user=f'user{user_index}', 
                hyper=model_architecture,
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.0006),
            )
            #add model weights to list        
            local_dense_moe_weights = local_model.get_weights()
            local_weight_list.append(local_dense_moe_weights)
        
            #clear session to free memory after each communication round
            K.clear_session()
        
        #to get the average over all the local model, we simply take the sum of the scaled weights
        average_weights_dense_moe = avg_weights_with_noise_fedprox(local_weight_list)
        #update global model 
        global_model.set_weights(average_weights_dense_moe)
        #Save global models
        global_model.save(cwd + f"/models/FL/Testing/cluster_{cluster_number}/{model_architecture}/FederatedRound{federated_round+1}")
        print("Saved Global models")

INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Testing/cluster_0/Dense_MoE_e4d8_d16/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Testing/cluster_0/Dense_MoE_e4d8_d16/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Testing/cluster_1/Dense_MoE_e4d8_d16/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Testing/cluster_1/Dense_MoE_e4d8_d16/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Testing/cluster_2/Dense_MoE_e4d8_d16/FederatedRound0\assets


...

INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Testing/cluster_2/Dense_MoE_e4d8_d16/FederatedRound3\assets


Saved Global models
Cluster 3:
User 3
User 8
User 9
User 14
User 19
User 27


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Testing/cluster_3/Dense_MoE_e4d8_d16/FederatedRound3\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Testing/cluster_3/Dense_MoE_e4d8_d16/FederatedRound3\assets


Saved Global models


In [10]:
#Evaluation - Version 1: Direkt
for cluster_number, users_in_cluster in cluster_users.items():
    print(f"Cluster {cluster_number}:")

    #Get global models weights
    global_model = tf.keras.models.load_model(cwd + f"/models/FL/Testing/cluster_{cluster_number}/{model_architecture}/FederatedRound{federated_rounds}", compile=False)

    #for idx, user in enumerate(df_array): 
    for user_index in users_in_cluster:
        print("User: ", user_index)
        for round in range(3):
            global_model = tf.keras.models.load_model(cwd + f"/models/FL/Testing/cluster_{cluster_number}/{model_architecture}/FederatedRound{federated_rounds}", compile=False)
            model = m1.build_soft_dense_moe_model(X_train[f'user{user_index}'], batch_size, horizon, dense_units, expert_units, num_experts, m1)
            model.set_weights(global_model.get_weights())
            
            histroy, user_results = mh.compile_fit_evaluate_model(
                model=model, 
                loss=loss, 
                metrics=metrics, 
                X_train=X_train[f'user{user_index}'],
                y_train = y_train[f'user{user_index}'], 
                max_epochs = 1, 
                batch_size=batch_size, 
                X_val=X_val[f'user{user_index}'], 
                y_val=y_val[f'user{user_index}'], 
                X_test=X_test[f'user{user_index}'], 
                y_test=y_test[f'user{user_index}'], 
                callbacks=callbacks, 
                user=f'user{user_index}', 
                hyper=model_architecture,
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.0006)
            )
            # Add the 'architecture' column from dense_user_results to dense_results
            all_results = pd.merge(all_results, user_results, how='outer')  

for idx in range(len(df_array)):
    new_row = {
        'architecture': model_architecture,
        'train_time': all_results[all_results["user"]==f"user{idx+1}"]["train_time"].mean(), 
        'avg_time_epoch' : all_results[all_results["user"]==f"user{idx+1}"]["avg_time_epoch"].mean(),
        'mse': all_results[all_results["user"]==f"user{idx+1}"]["mse"].mean(),
        'mse_std' : all_results[all_results["user"]==f"user{idx+1}"]["mse"].std(),
        'rmse': all_results[all_results["user"]==f"user{idx+1}"]["rmse"].mean(),
        'rmse_std' : all_results[all_results["user"]==f"user{idx+1}"]["rmse"].std(),
        'mape': all_results[all_results["user"]==f"user{idx+1}"]["mape"].mean(),
        'mape_std' : all_results[all_results["user"]==f"user{idx+1}"]["mape"].std(),
        'mae': all_results[all_results["user"]==f"user{idx+1}"]["mae"].mean(),
        'mae_std' : all_results[all_results["user"]==f"user{idx+1}"]["mae"].std(),
    }
    results.loc[len(results)] = new_row


Cluster 0:
User:  1
User:  2
User:  4
User:  5
User:  7
User:  10
User:  11
User:  12
User:  15
User:  16
User:  17
User:  21
User:  22
User:  23
User:  24
User:  25
User:  28
User:  30
Cluster 1:
User:  29
Cluster 2:
User:  6
User:  13
User:  18
User:  20
User:  26
Cluster 3:
User:  3
User:  8
User:  9
User:  14
User:  19
User:  27


In [11]:
df_evaluation = results.tail(30)
df_evaluation.reset_index(inplace=True)
df_results = pd.DataFrame(columns=["cluster", "architecture", "time","mse", "rmse", "mape", "mae"])

for cluster_number, users_in_cluster in cluster_users.items():
    #print(f"Cluster {cluster_number}:")
    
    cluster = f"cluster_{cluster_number}"
    architecture = df_evaluation["architecture"][0]
    time = 0
    mse = 0
    rmse = 0
    mape = 0
    mae = 0
    
    # Filter and print the users in the current cluster
    for user_index in users_in_cluster:
        time += df_evaluation["train_time"][user_index-1]
        mse += df_evaluation["mse"][user_index-1]
        mae += df_evaluation["mae"][user_index-1]
        rmse += df_evaluation["rmse"][user_index-1]
        mape += df_evaluation["mape"][user_index-1]
        
    num_users_in_cluster = len(users_in_cluster)
    new_row = {
        'cluster': cluster,
        'architecture': architecture, 
        'time' : time / num_users_in_cluster,
        'mse': mse / num_users_in_cluster,
        "rmse" : rmse / num_users_in_cluster,
        "mape": mape / num_users_in_cluster,
        'mae':  mae / num_users_in_cluster,
    }
    df_results.loc[len(df_results)] = new_row
    
moe_1 = df_results
print("Mean: ", df_results["mse"].mean())
df_results

Mean:  0.01869731661087523


,cluster,architecture,time,mse,rmse,mape,mae
0,cluster_0,Dense_MoE_e4d8_d16,1.378589,0.015697,0.123308,310467.102358,0.075675
1,cluster_1,Dense_MoE_e4d8_d16,1.506782,0.018375,0.135547,494650.666667,0.096580
2,cluster_2,Dense_MoE_e4d8_d16,1.401499,0.021122,0.144063,138798.159896,0.098415
3,cluster_3,Dense_MoE_e4d8_d16,1.371836,0.019595,0.139187,74041.048158,0.085833
